# PySide6 学习笔记

## 第一课：创建第一个窗口

**关键知识点:**

*   **`QApplication`**: 每个应用的核心，必须且只能创建一个实例
*   **`QWidget`**: 所有UI元素的基类，可以作为一个空白窗口使用
*   **`.show()`**: 使控件（包括窗口）可见的方法
*   **`app.exec()`**: 启动应用程序的事件循环，让程序等待用户操作

In [ ]:
# 1. 导入必要的模块
import sys
from PySide6.QtWidgets import QApplication, QWidget

# 2. 创建一个应用程序实例
# sys.argv 允许从命令行传递参数给应用程序
app = QApplication(sys.argv)

# 3. 创建一个窗口对象
# QWidget 是最基础的空白容器控件
window = QWidget()
window.setWindowTitle("我的第一个窗口") # 设置窗口标题
window.resize(300, 200)       # 设置窗口初始尺寸

# 4. 显示窗口
# 默认创建的控件是不可见的，必须调用 show()
window.show()

# 5. 启动事件循环
# 程序会在此处阻塞，直到窗口关闭，然后安全退出
sys.exit(app.exec())

## 第二课：面向对象的方式构建窗口

**关键知识点:**

*   **继承**: 自定义窗口类应继承自 `QWidget` 或其他窗口类型。
*   **`__init__`**: 构造函数，用于初始化窗口。
*   **`super().__init__()`**: 在构造函数中**必须首先**调用父类的构造函数。
*   **`if __name__ == "__main__":`**: 保护程序入口，使UI类可以被复用。

In [ ]:
import sys
from PySide6.QtWidgets import QApplication, QWidget

# 定义一个继承自 QWidget 的新类
class MainWindow(QWidget):
    # 类的构造函数
    def __init__(self):
        # a. 调用父类的构造函数，完成底层初始化
        super().__init__()
        # b. 调用一个专门的方法来设置 UI，保持构造函数整洁
        self.setup_ui()

    def setup_ui(self):
        # self 指代实例对象自身，通过 self 调用继承来的方法
        self.setWindowTitle("我的面向对象窗口")
        self.resize(800, 600)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()  # 实例化我们自定义的窗口类
    window.show()
    sys.exit(app.exec())

## 第三课：添加控件与父子关系

**关键知识点:**

*   **父子关系**: 在创建子控件时传入父控件 (`QLabel("text", self)`)。
*   **自动内存管理**: 父控件销毁时，所有子控件自动销毁。
*   **相对坐标**: 子控件的位置是相对于父控件的左上角。
*   **绝对定位 `.move(x, y)`**: 手动指定控件位置，但无法适应窗口缩放。

In [ ]:
import sys
from PySide6.QtWidgets import QApplication, QWidget, QLabel

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setup_ui()

    def setup_ui(self):
        self.setWindowTitle("控件与父子关系")
        self.resize(400, 300)
        
        # 创建一个 QLabel 实例
        # 第二个参数 self 指定了它的父控件是 MainWindow 实例
        # 这使得 label 会在 window 上显示，并随 window 关闭而销毁
        label1 = QLabel("Hello, World!", self)
        label1.move(150, 130)
        
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec())

## 第四课：布局管理器

**关键知识点:**

*   **布局管理器**: 自动管理控件的大小和位置。
*   **`QVBoxLayout`**: 垂直布局，从上到下排列控件。
*   **`QHBoxLayout`**: 水平布局，从左到右排列控件。
*   **使用三部曲**:
    1.  创建布局实例 (`layout = QVBoxLayout()`)。
    2.  将控件添加进布局 (`layout.addWidget(...)`)。
    3.  将布局应用到窗口 (`self.setLayout(layout)`)。
*   **布局嵌套**: 使用 `addLayout()` 将一个布局添加到另一个布局中，构建复杂界面。

In [ ]:
import sys
from PySide6.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QHBoxLayout

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setup_ui()

    def setup_ui(self):
        self.setWindowTitle("布局管理器")
        self.resize(300, 200)

        label1 = QLabel("左")
        label2 = QLabel("右")

        # 1. 创建一个水平布局实例
        layout = QHBoxLayout()
        # 2. 将控件添加进布局
        layout.addWidget(label1)
        layout.addWidget(label2)
        # 3. 将此布局应用到窗口
        self.setLayout(layout)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec())

## 第五课：交互与信号槽

我们引入了第一个交互式控件 `QPushButton`，并学习了Qt的事件处理核心机制——信号与槽。

**关键知识点:**

*   **信号 (Signal)**: 控件在特定事件发生时发射的“广播”，如 `button.clicked`。
*   **槽 (Slot)**: 一个用于响应信号的Python函数或方法。
*   **连接 (Connect)**: 使用 `signal.connect(slot)` 将信号和槽关联起来。
*   **实例属性 (`self.xxx`)**: 将控件保存为实例属性，以便在类的不同方法中访问。

In [ ]:
import sys
from PySide6.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QPushButton

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        # 将需要在多个方法中访问的数据设为实例属性
        self.click_count = 0
        self.setup_ui()

    def setup_ui(self):
        self.setWindowTitle("按钮与信号槽")
        # 将需要在槽函数中修改的控件也设为实例属性
        self.info_label = QLabel("你还没有点击按钮。")
        button = QPushButton("点我!")
        
        # 核心：将按钮的 clicked 信号连接到 on_button_clicked 槽方法
        # 注意：这里传递的是方法对象 self.on_button_clicked，没有括号
        button.clicked.connect(self.on_button_clicked)

        layout = QVBoxLayout()
        layout.addWidget(self.info_label)
        layout.addWidget(button)
        self.setLayout(layout)
    
    # 定义槽函数
    def on_button_clicked(self):
        self.click_count += 1
        self.info_label.setText(f"按钮被点击了 {self.click_count} 次。")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec())

## 第六课：获取用户输入

我们学习了如何使用 `QLineEdit` 来获取用户的单行文本输入。

**关键知识点:**

*   **`QLineEdit`**: 单行文本输入框。
*   **`.text()`**: 获取输入框当前内容的方法。
*   **`.setPlaceholderText()`**: 设置当输入框为空时显示的灰色提示文本。
*   **`returnPressed`**: 用户在输入框中按下回车键时发射的信号。

In [ ]:
import sys
from PySide6.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QLineEdit

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setup_ui()

    def setup_ui(self):
        self.setWindowTitle("获取用户输入")
        self.info_label = QLabel("请在下方输入你的名字:")
        self.name_input = QLineEdit()
        self.name_input.setPlaceholderText("请在此输入...")
        
        greet_button = QPushButton("问候")
        
        layout = QVBoxLayout()
        layout.addWidget(self.info_label)
        layout.addWidget(self.name_input)
        layout.addWidget(greet_button)
        self.setLayout(layout)

        # 连接按钮的点击信号和输入框的回车信号到同一个槽
        greet_button.clicked.connect(self.greet)
        self.name_input.returnPressed.connect(self.greet)

    def greet(self):
        # 使用 .text() 方法获取输入内容
        name = self.name_input.text()
        if name:
            self.info_label.setText(f"你好, {name}!")
        else:
            self.info_label.setText("请输入你的名字！")

# ... (if __name__ == "__main__": 部分保持不变)
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec())

## 第七课：多选项选择

**关键知识点:**

*   **`QComboBox`**: 下拉列表控件。
*   **`.addItem()` / `.addItems()`**: 向下拉框中添加一个或多个选项。
*   **带参数的信号**: 如 `currentTextChanged`，它在发射时会携带数据（当前文本）。
*   **带参数的槽**: 槽函数可以定义参数来接收信号传来的数据。
*   **`.setCurrentText()`**: 设置默认显示的选项。

In [ ]:
import sys
from PySide6.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QComboBox

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setup_ui()

    def setup_ui(self):
        self.setWindowTitle("下拉选择框")
        self.language_combo = QComboBox()
        
        # 使用 addItems 批量添加选项
        languages = ["Python", "C++", "Java", "Go"]
        self.language_combo.addItems(languages)
        
        # 设置默认选项
        self.language_combo.setCurrentText("Python")
        
        self.result_label = QLabel()

        # 连接 currentTextChanged 信号到槽函数
        # 这个信号会把新选中的文本作为参数传给槽
        self.language_combo.currentTextChanged.connect(self.on_language_changed)
        
        # 首次运行时手动调用一次槽，以初始化 result_label 的文本
        self.on_language_changed(self.language_combo.currentText())

        layout = QVBoxLayout()
        layout.addWidget(self.language_combo)
        layout.addWidget(self.result_label)
        self.setLayout(layout)
        
    # 槽函数定义了一个 text 参数来接收信号传来的数据
    def on_language_changed(self, text):
        if text:
            self.result_label.setText(f"你选择了: {text}")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec())